In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-data/heart-disease.csv


In [2]:
import os

# Create the folder only if it doesn't exist
if not os.path.exists("heart_disease_project"):
    os.mkdir("heart_disease_project")


In [3]:
cd heart_disease_project

/kaggle/working/heart_disease_project


## predicting heart disease using machine learning 

what approach we are going to take:
1. problem defination
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

## Problem defination 
>Given clinical parameters about a patient , can we predict whether the patient has a heart disease or not.

## Data 
>the data is taken from the kaggle dataset heart disease data . 

## Evaluation 
>if we could get to a 95% accuracy at predicting whether a patient has a heart disease or not , we will consider the model.


## Features 
 * "age",
 * "sex",
 *  "cp",          # chest pain type
 *  "trestbps",    # resting blood pressure
 *  "chol",        # serum cholesterol
 *  "fbs",         # fasting blood sugar
 *  "restecg",     # resting electrocardiographic results
 *  "thalach",     # maximum heart rate achieved
 *  "exang",       # exercise induced angina
 *  "oldpeak",     # ST depression induced by exercise
 *  "slope",       # slope of the peak exercise ST segment
 *  "ca",          # number of major vessels colored by fluoroscopy
 *  "thal",        # thalassemia
 * "target"       # diagnosis of heart disease

## Preparing the tools 


In [4]:
# we are going to use pandas , numpy , matplotlib for data analysis and maipulation

#Regualar EDA (exploratory data analysis) and plotting libraries 

import pandas as pd 
import numpy as np 
%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns 

# models from sklearn 

from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 

# Model Evaluation 
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV

# A confusion matrix is a table that shows how well a classification
# model is performing by comparing the actual labels vs the predicted labels.

from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.metrics import precision_score , recall_score , f1_score
from sklearn.metrics import RocCurveDisplay




In [5]:

heart_disease = pd.read_csv("/kaggle/input/heart-disease-data/heart-disease.csv")

In [6]:
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
heart_disease.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64